In [1]:
#Please put the code in the same directory with 'SimplifiedFashionDataset'
import os
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy import asarray
import torch.optim as optim
Jumpsuit = 'SimplifiedFashionDataset/Jumpsuit'
Jumpsuit_label = np.full(len(os.listdir(Jumpsuit)), 0)
Dress = 'SimplifiedFashionDataset/Dress'
Dress_label = np.full(len(os.listdir(Dress)), 1)
Skirt = 'SimplifiedFashionDataset/Skirt'
Skirt_label = np.full(len(os.listdir(Skirt)), 2)
Tee = 'SimplifiedFashionDataset/Tee'
Tee_label = np.full(len(os.listdir(Tee)), 3)
label_name = {0:'Jumpsuit', 1:'Dress', 2:'Skirt', 3:'Tee'}
file_name = os.listdir(Jumpsuit) + os.listdir(Dress) + os.listdir(Skirt) + os.listdir(Tee)
label = np.concatenate((Jumpsuit_label, Dress_label, Skirt_label, Tee_label))
len(label)

129927

In [2]:
#I use CNN to classify the images in PyTorch
class myClassifier(nn.Module):
    def __init__(self):
        super(myClassifier, self).__init__()
        self.block1 = self.conv_block(c_in=3, c_out=128, dropout=0.1, kernel_size=5, stride=1, padding=2)
        self.block2 = self.conv_block(c_in=128, c_out=64, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.block3 = self.conv_block(c_in=64, c_out=32, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.lastcnn = nn.Conv2d(in_channels=32, out_channels=4, kernel_size=75, stride=1, padding=0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    def forward(self, x):
        x = self.block1(x)
        x = self.maxpool(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.maxpool(x)
        x = self.lastcnn(x)
        return x
    def conv_block(self, c_in, c_out, dropout, **kwargs):
        seq_block = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU(),
            nn.Dropout2d(p=dropout)
        )
        return seq_block
net = myClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [3]:
#Considering I don't have GPU and running time is too long, I only randomly use 200 images as training dataset
def train(net):
    for epoch in range(1):
        for index in np.random.randint(0, len(label), 200):
            optimizer.zero_grad()
            image_path = 'SimplifiedFashionDataset/'+ label_name[label[index]] + '/' + file_name[index]
            img = Image.open(image_path)
            new = img.resize((300, 300))
            data = asarray(new)
            outputs = net(torch.from_numpy(-1+(2/255)*data).float().view(1, 3, 300, 300))
            loss = criterion(outputs, torch.tensor([[[label[index]]]], dtype=torch.long))
            loss.backward()
            optimizer.step()
train(net)

In [6]:
#I use 100 images as test dataset and 68 of them are predicted correctly
#If I train them with more dataset, more time and use GPU to train, I believe the accuracy must be much higher
correct = 0
with torch.no_grad():
    for index in np.random.randint(0, len(label), 100):
        image_path = 'SimplifiedFashionDataset/'+ label_name[label[index]] + '/' + file_name[index]
        img = Image.open(image_path)
        new = img.resize((300, 300))
        data = asarray(new)
        outputs = net(torch.from_numpy(-1+(2/255)*data).float().view(1, 3, 300, 300))
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == label[index]).sum()
correct

tensor(68)